In [ ]:
# In data validation step , you will check that whether the columns in the data that is going to be used for the training of the model is valid . Here the validation will be limited to the column names verfication . However this validation step can be more extensive wherein you can also check if the column data type is valid or not .For this verification of datatype of each column will be done. 
# For this schema.yml file is used where user pre-defines the column names and the datatype for the dataset coming for the model training. 

In [2]:
%pwd

'd:\\WineQuality_EndToEndDataScienceProject\\WineQualityDataScienceProject\\research'

In [3]:
import os
os.chdir("../")
%pwd

'd:\\WineQuality_EndToEndDataScienceProject\\WineQualityDataScienceProject'

In [4]:
import pandas as pd 
data = pd.read_csv("artifacts/data_ingestion/winequality-red.csv")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [12]:
from src.WineQualityPrediction.utils.common import read_yaml , create_directories
from src.WineQualityPrediction.entity.config_entity import DataValidationConfig 
from src.WineQualityPrediction.constants import config_file_path , params_file_path , schema_file_path

class ConfigurationManager:
    def __init__(self ,
                 config_filepath = config_file_path,
                 schema_filepath = schema_file_path,
                 params_filepath = params_file_path):
        
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS
        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            DataToValidateLoc=config.dataset_location,
            StatusFileFolder=config.root_dir,
            StatusFileName=config.status_file,
            AllSchema=schema
        )
        return data_validation_config
    




In [13]:
data.shape

(1599, 12)

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [63]:
from src.WineQualityPrediction.constants import schema_file_path
from src.WineQualityPrediction.utils.common import read_yaml , create_directories

schema = read_yaml(schema_file_path)
schema_cols = schema.COLUMNS.keys()
schema_cols_dtype = list(schema.COLUMNS.values())
class DataValidation:
    def __init__(self , config : DataValidationConfig) :
        self.config = config 

    def get_data_validation_status(self):
        try :
            validation_status = None
            data = self.config.DataToValidateLoc
            data = pd.read_csv(data)
            data_columns = list(data.columns)
            dtypes_list = [col.name for col in list(data.dtypes)]
            
            for i,col in enumerate(data_columns):
                if col not in schema_cols:
                    validation_status = False
                    with open(self.config.StatusFileName , 'w') as file:
                        file.write(f"Validation Status : {validation_status}")
                    break
                if dtypes_list[i] != schema_cols_dtype[i]:
                    validation_status = False
                    with open(self.config.StatusFileName , 'w') as file:
                        file.write(f"Validation Status : {validation_status}")
                    break

            if validation_status == None:
                validation_status = True
                with open(self.config.StatusFileName , 'w') as file:
                        file.write(f"Validation Status : {validation_status}")

            

            return validation_status
        
        except Exception as e:
            raise e
        
        
    
            
        

[2025-04-10 23:34:04,277] : INFO : common : yaml file: schema.yml loaded successfully


In [62]:
list(data.columns)

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality']

In [40]:
from src.WineQualityPrediction.constants import schema_file_path
schema = read_yaml(schema_file_path)
list(schema.COLUMNS.values())


[2025-04-10 23:24:19,542] : INFO : common : yaml file: schema.yml loaded successfully


['float64',
 'float64',
 'float64',
 'float64',
 'float64',
 'float64',
 'float64',
 'float64',
 'float64',
 'float64',
 'float64',
 'int64']

In [57]:
[col.name for col in list(data.dtypes)]

['float64',
 'float64',
 'float64',
 'float64',
 'float64',
 'float64',
 'float64',
 'float64',
 'float64',
 'float64',
 'float64',
 'int64']

In [64]:
try :
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config = data_validation_config)
    validation_status = data_validation.get_data_validation_status()
    print(f"Validation Status : {validation_status}")

except Exception as e :
    raise e

[2025-04-10 23:34:09,031] : INFO : common : yaml file: config\config.yml loaded successfully
[2025-04-10 23:34:09,034] : INFO : common : yaml file: schema.yml loaded successfully
[2025-04-10 23:34:09,038] : INFO : common : yaml file: params.yml loaded successfully
[2025-04-10 23:34:09,041] : INFO : common : Created directory at: artifacts
[2025-04-10 23:34:09,042] : INFO : common : Created directory at: artifacts/data_validation
Validation Status : True
